In [ ]:
import time

import pandas as pd

from etrade_track.z_helper import read_xlsx_codes

#
codes0 = read_xlsx_codes('short.xls')
print(codes0)

In [ ]:
codes1 = read_xlsx_codes('shortz.xls')
print(codes1)


In [ ]:
codes0 = read_xlsx_codes('short.xls')
print(codes0)

# rsi price 跟踪 short
from etrade_track.z_helper import *
from etrade_track.z_algo_rsi import *
from MyTT import *

for c in codes0:
    download_rsi_data([c], '60m', 120)
    pklfile = f'temp/{c}_60m.pkl'
    df = pd.read_pickle(pklfile)
    rsi = RSIP(df)

    r = rsi[-1]
    close = df['close'].values[-1]
    delta = round((close - r) / close * 100, 2)

    if abs(delta) < 10:
        print(f'{c} {db_id_to_name(c)} {close} {r:.2f} {delta}')
    # print(f'{c} {db_id_to_name(c)} {close} {r:.2f} {delta}')
    pass

In [ ]:
from stockbase.stock_db import *
from etrade_track.z_helper import *
from etrade_track.z_algo_rsi import *
from MyTT import *

stocks = db_select_tscodes()

print(stocks)


download_rsi_data(['sz000001'], '1w', 104)

c = 'sz000001'
pklfile = f'temp/{c}_1w.pkl'
df = pd.read_pickle(pklfile)
print(df)

In [ ]:
with open('rsi_week.csv', 'w') as fs:
    for c in stocks:
        try:
            download_rsi_data([c], '1w', 104)
            pklfile = f'temp/{c}_1w.pkl'
            if not os.path.exists(pklfile):
                print(f'{c} data error')
                continue
                pass
            df = pd.read_pickle(pklfile)

            rsi = RSIP(df)
            r = rsi[-1]
            close = df['close'].values[-1]
            delta = round((close - r) / close * 100, 2)

            if delta < 1:
                # 周线接近恐慌
                rsistr = f'{c} {db_id_to_name(c)} {close:.2f} {r:.2f} {delta:.2f}'
                print(rsistr)
                fs.write(rsistr.replace(' ', ',')+'\n')
        except Exception as ex:
            print(c, ex)
            break
            pass
    pass


In [ ]:
# rsi price 计算

def RSIP(df):
    C = df['close']
    C = C.dropna()
    C = pd.to_numeric(C)
    C = np.array(C)
    Z = SMA(MAX(C - REF(C, 1), 0), 6, 1)
    Y = SMA(MAX(ABS(C - REF(C, 1)), 0), 6, 1)
    R1 = Y * 5 - Z * 25 + C
    R2 = Y * 5 / 4 - Z * 25 / 4 + C
    R0 = IF(Z / Y < 0.2, R2, R1)
    R = REF(R0, 1)
    return R


c = 'sz002750'
download_rsi_data([c], '60m', 60)
pklfile = f'temp/{c}_60m.pkl'
df = pd.read_pickle(pklfile)

rsi = RSIP(df)
# print(rsi)
print('\n'.join([str(r) for r in rsi[-20:]]))




In [ ]:
# rsi 买卖回测测试

import datetime
import math

from stockbase.stock_db import *

from MyTT import *


def getday(dt):
    dt = datetime.datetime.utcfromtimestamp(dt.astype('O') / 1e9)
    # print(dt.day)
    return dt.day
    pass


def rsi_fuck_df(df, si=-20, ei=0, s0pct=3, s1pct=5, enrsi=False, dpct=0.5, dcount=3):
    # 数据准备
    # df 60m数据
    # rsi
    rsiv = RSI(df['close'], 6)
    # rsi 价格
    rsi = RSIP(df)
    # 日期
    date = df['date'].values

    op = []  # 操作记录
    hold = 0  #  买入状态
    holddate = 0  # 买入日期
    holdp = 0  # 买入价格
    hdelta = 0  # 冲高监控最小单位
    holdhigh = 0  # 最高价格

    selldate = 0
    sell0p = 0  # 止损价格
    sell1p = 0  # 止盈价格

    # 回测范围
    rng = range(si, ei, 1)
    for ri in rng:
        low = df['low'].values[ri]
        high = df['high'].values[ri]
        close = df['close'].values[ri]
        rsip = round(rsi[ri], 2)

        if hold == 0:
            # 买入判断
            # empty wait to buy
            # if low < rsip and rsiv[ri] <= 20:
            if low < rsip:
                # close10 = max(df['high'].values[ri - 10:ri])
                # delta = (close - close10) / close10 * 100
                # if delta > -2 or delta < -6:
                #     # 相对前十个小时最高收盘价
                #     continue
                #     pass
                if enrsi and rsiv[ri] > 20:
                    # 启用 rsi 判断 且 rsi 未触发
                    # 跳过
                    continue
                    pass
                # buy
                hold = 1
                holdp = round(rsip, 2)
                holdhigh = holdp
                hdelta = holdhigh * dpct / 100
                holddate = date[ri]
                sell1p = round(rsip * (100 + s1pct) / 100, 2)
                sell0p = round(rsip * (100 - s0pct) / 100, 2)
                op.append([1, rsip, date[ri]])
                # print(f'buy:{rsip:.2f} {date[ri]}')
                # print('sell0p', sell0p)
                # print('sell1p', sell1p)

        elif hold > 0 and getday(date[ri]) != getday(holddate):
            # 隔日 卖出判断
            # wait to sell
            if low < sell0p:
                # 股价超过止损线
                # 止损
                if hold > 0:
                    op.append([-1, sell0p, date[ri]])
                    # print(f'sell0:{sell0p:.2f} {date[ri]}')
                    hold = -1
            # elif high >= sell1p:
            #     # 股价超过止盈线
            #     # 止盈
            #     # if hold > 0:
            #     #     op.append([0, sell1p, date[ri]])
            #     #     # print(f'sell1:{sell1p:.2f} {date[ri]}')
            #     #     hold = -1
            #     pass
            elif high > holdp:
                # 冲高回落止盈
                # 股价上涨更新高点
                if high > holdhigh:
                    # 冲高格子计数
                    holdhigh = high
                    hdelta = holdhigh * dpct / 100
                    # print(f'update high:{holdhigh}')
                # 冲高
                if holdhigh > 0:
                    # 回落止盈价格
                    highp = holdhigh - dcount * hdelta
                    # 高点回落
                    if close < highp:
                        if hold > 0:
                            if highp > holdp:
                                # 回撤至固定点位卖出
                                op.append([0, highp, date[ri]])
                                # print(f'sell1:{highp:.2f} {date[ri]}')
                            else:
                                # 回撤至原价卖出
                                op.append([-1, holdp, date[ri]])
                                # print(f'sell0:{holdp:.2f} {date[ri]}')
                            selldate = date[ri]
                            hold = -1

            if ri == ei - 1:
                # 最后强制卖出
                if close > holdp:
                    # 止盈
                    op.append([0, close, date[ri]])
                    # print(f'sell1:{highp:.2f} {date[ri]}')
                else:
                    # 止损
                    op.append([-1, close, date[ri]])
                    # print(f'sell0:{holdp:.2f} {date[ri]}')
                hold = -1
                pass

        if hold == -1:
            #print('op finish')
            break
            # if getday(date[ri]) != getday(selldate):
            #     # wait next day to continue buy
            #     hold = 0
            pass

    pct = 0
    if len(op) == 2:
        if op[0][0] == 1:
            p0 = op[0][1]
            p1 = op[1][1]
            pct = round((p1 - p0) / p0 * 100, 2)

    return op, pct


def formatop(ops):
    if len(ops) == 0:
        return ',,,,,'
    if len(ops) == 1:
        opstr = ','.join([','.join([str(o) for o in op]) for op in ops])
        return opstr + ',,,'

    opstr = ','.join([','.join([str(o) for o in op]) for op in ops])
    #print(opstr)
    return opstr


#rsi价格买卖回测

from MyTT import *
from etrade_track.z_helper import *

file = 'short.xls'

code = read_xlsx_codes(file)
# file = 'shortz.txt'
# code = read_txt_code(file)
print(code)

buysellcount = 0
sell1count = 0
sell1pct = 0
sell0count = 0
sell0pct = 0
pctsum = 0

#止损
sell0pctval = 3
#止盈
sell1pctval = 3

deltapct = 0.5
deltacount = 4

# ssi = -44
# eei = -24
enablersi = False

ssi = -20
eei = ssi + 20

datestr = ''

with open(f'{file}_backtest.csv', 'w') as f:
    for c in code:
        c = str.lower(c)
        # print(f'{c} start ...')
        try:
            download_rsi_data([c], '60m', 4 * 5 * 2)
            df = pd.read_pickle(f'temp/{c}_60m.pkl')
            # ma60 = MA(df['close'], 90)
            if df.shape[0] < abs(ssi):
                continue
            # if ma60[ssi]>df['close'].values[ssi]:
            #     continue

            if datestr == '':
                datestr = df['date'].values[ssi]

            ops, pct = rsi_fuck_df(df,
                                   si=ssi,
                                   ei=eei,
                                   s0pct=sell0pctval,
                                   s1pct=sell1pctval,
                                   enrsi=enablersi,
                                   dpct=deltapct,
                                   dcount=deltacount)
            if len(ops) == 2:
                pctsum = pctsum + pct
                buysellcount = buysellcount + 1
                if pct > 0:
                    sell1count = sell1count + 1
                    sell1pct = sell1pct + pct
                else:
                    sell0count = sell0count + 1
                    sell0pct = sell0pct + pct

                csvstr = f'{c},{db_id_to_name(c)},{pct:.2f},{formatop(ops)}\n'
                print(csvstr)
                f.write(csvstr)
                # print(f'{c} {index} finish {pct}...\n')
        except Exception as ex:
            print(c, ex)
            pass

print(f'\n\nbacktest finish {datestr}')
# print(datetime.datetime.now())

buysellcount = 1e-3 if buysellcount == 0 else buysellcount

p1 = round(sell1count / buysellcount * 100, 2)
p0 = round(sell0count / buysellcount * 100, 2)
ev = p1 / 100 * sell1pctval - p0 / 100 * sell0pctval

print(f'sell0pct:{sell0pctval}')
print(f'sell1pct:{sell1pctval}')
print(f'total:{buysellcount}')
print(f'sell1count:{sell1count} p1:{p1:.2f}')
print(f'sell0count:{sell0count} p0:{p0:.2f}')
print(f'pctsum:{pctsum}')
print(f'avgsum:{pctsum / buysellcount:.2f}')
print(f'sell1pct:{sell1pct:.2f}')
print(f'sell0pct:{sell0pct:.2f}')
print(f'ev:{ev:.2f}')



In [ ]:
from MyTT import *
from etrade_track.z_helper import *

# code = read_xlsx_codes('short.xls')
# file = 'shortz.txt'
# code = read_txt_code(file)

from etrade_track.z_helper import read_xlsx_codes

file = 'table0110'
df = pd.read_csv('../temp/Table0110.xls', encoding='gbk', sep='\t')
df['总市值'] = pd.to_numeric(df['总市值'], errors='coerce')
df['总市值'] = df['总市值'] / 1e8
df = df[df['总市值'] < 1000]
df = df[df['总市值'] > 500]

code = df.iloc[:, 1]
print(len(code))

print(code)

for i in range(1):
    buysellcount = 0
    sell1count = 0
    sell1pct = 0
    sell0count = 0
    sell0pct = 0
    pctsum = 0

    #止损
    sell0pctval = 3
    #止盈
    sell1pctval = 5

    deltapct = 0.5
    deltacount = 4

    # ssi = -44
    # eei = -24
    enablersi = False

    ssi = -20 * (i + 1) - 4
    eei = ssi + 20 - 4

    datestr = ''

    with open(f'{file}_backtest_{i}.csv', 'w') as f:
        index = 0
        for c in code:
            index = index + 1
            c = str.lower(c)
            if c.startswith('sh688'):
                continue
            # print(f'{c} start ...')
            try:
                if not os.path.exists(f'temp/{c}_60m.pkl'):
                    download_rsi_data([c], '60m', 4 * 5 * 50)
                df = pd.read_pickle(f'temp/{c}_60m.pkl')
                # ma60 = MA(df['close'], 90)
                if df.shape[0] < abs(ssi):
                    continue
                # if ma60[ssi]>df['close'].values[ssi]:
                #     continue

                if datestr == '':
                    datestr = df['date'].values[ssi]

                ops, pct = rsi_fuck_df(df,
                                       si=ssi,
                                       ei=eei,
                                       s0pct=sell0pctval,
                                       s1pct=sell1pctval,
                                       enrsi=enablersi,
                                       dpct=deltapct,
                                       dcount=deltacount)
                if len(ops) == 2:
                    pctsum = pctsum + pct
                    buysellcount = buysellcount + 1
                    if pct > 0:
                        sell1count = sell1count + 1
                        sell1pct = sell1pct + pct
                    else:
                        sell0count = sell0count + 1
                        sell0pct = sell0pct + pct

                    csvstr = f'{c},{db_id_to_name(c)},{pct:.2f},{formatop(ops)}\n'
                    # print(csvstr)
                    f.write(csvstr)
                    # print(f'{c} {index} finish {pct}...\n')
            except Exception as ex:
                print(c, ex)
                pass

    print(f'\n\nbacktest finish {i} {datestr}')
    # print(datetime.datetime.now())

    buysellcount = 1e-3 if buysellcount == 0 else buysellcount

    p1 = round(sell1count / buysellcount * 100, 2)
    p0 = round(sell0count / buysellcount * 100, 2)
    ev = p1 / 100 * sell1pctval - p0 / 100 * sell0pctval

    print(f'sell0pct:{sell0pctval}')
    print(f'sell1pct:{sell1pctval}')
    print(f'total:{buysellcount}')
    print(f'sell1count:{sell1count} p1:{p1:.2f}')
    print(f'sell0count:{sell0count} p0:{p0:.2f}')
    print(f'pctsum:{pctsum}')
    print(f'avgsum:{pctsum / buysellcount:.2f}')
    print(f'sell1pct:{sell1pct:.2f}')
    print(f'sell0pct:{sell0pct:.2f}')
    print(f'ev:{ev:.2f}')

    with open('backtest_opt.csv', 'a+') as fs:
        fs.write(f'{datestr},')
        # fs.write(f'deltapct,{deltapct},')
        # fs.write(f'deltacount,{deltacount},')
        # fs.write(f'sell0pct,{sell0pctval},')
        # fs.write(f'sell1pct,{sell1pctval},')
        fs.write(f'total,{buysellcount},')
        fs.write(f'sell1count,{sell1count} p1,{p1:.2f},')
        fs.write(f'sell0count,{sell0count} p0,{p0:.2f},')
        fs.write(f'pctsum,{pctsum:.2f},')
        fs.write(f'avgsum,{pctsum / buysellcount:.2f},')
        fs.write(f'sell1pct,{sell1pct:.2f},')
        fs.write(f'sell0pct,{sell0pct:.2f},')
        fs.write(f'ev,{ev:.2f}\n')

print('back test finish')

In [ ]:
codes = read_txt_code('short - 1.txt')
print(codes)

In [ ]:

code = 'sz002245'

pklfile = f'temp/{code}_60m.pkl'

df = pd.read_pickle(pklfile)

ops, pct = rsi_fuck_df(df)

for op in ops:
    print(op)
print(pct)


In [ ]:
from stockbase.stock_db import *

import pandas as pd

from MyTT import *

stocks = db_select_tscodes()
print(stocks)
print(len(stocks))
stocks = [s for s in stocks if 100>db_select_pe(int(s[2:]))>0]
print(len(stocks))


selectstocks = []
for s in stocks:
    df = pd.read_pickle(f'../etrade_track/temp/{s}_1w.pkl')
    if df.shape[0]<2:
        continue
    closes = df['close'].values
    bh,bm,bl = BOLL(closes[-60:])
    highs = df['high'].values
    # boll 突破
    if closes[-1]>bh[-2] and highs[-2] < bh[-2]:
        info = f'close:{closes[-1]:.2f} bollhigh:{bh[-1]:.2f}'
        selectstocks.append([s, info])
        pass


for s in selectstocks:
    code = s[0]
    info =s[1]
    print(code, db_id_to_name(code), info)

print('')
print(f'found {len(selectstocks)} in {len(stocks)}')

In [ ]:
from stockbase.stock_db import *

import pandas as pd

from MyTT import *

stocks = db_select_tscodes()
print(stocks)
print(len(stocks))
# stocks = [s for s in stocks if 100>db_select_pe(int(s[2:]))>0]
# print(len(stocks))
#

selectstocks = []
for s in stocks:
    df = pd.read_pickle(f'../etrade_track/temp/{s}_1w.pkl')
    if df.shape[0]<3:
        continue
    closes = df['close'].values
    pct = (closes[-1]-closes[-2])/closes[-1]*100
    pct1 = (closes[-2]-closes[-3])/closes[-3]*100
    if pct>10 and pct1<5:
        info = f'pct:{pct:.2f}'
        selectstocks.append([s,info])


for s in selectstocks:
    code = s[0]
    info =s[1]
    print(code, db_id_to_name(code), info)

print('')
print(f'found {len(selectstocks)} in {len(stocks)}')